## Introdução as *RNNs* biblioteca Keras

Neste notebook iremos praticar a sintaxe básica para treinar uma rede neural artificial recorrente utilizando as bibliotecas [Keras](https://keras.io/) e [Tensorflow](https://www.tensorflow.org/).

<br/><br/>
Embora nós iremos importar diretamente apenas objetos e funções da biblioteca Keras, a biblioteca tensorflow é necessária para que o treinamento das redes neurais seja feito.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random as python_random

np.random.seed(42)
python_random.seed(42)
tf.random.set_seed(42)
tf.config.list_physical_devices()

A biblioteca Keras possui diversos datasets pré-processados para facilitar a execução de testes. Além disso, essa disponibilização facilita o aprendizado de sua utilização.

<br/><br/>

Abaixo iremos importar o dataset IMDB. Este é um dataset utilizado para *classificação de sentimento* em reviews sobre filmes. Esses reviews podem ser classificados como positivos ou negativos. Este dataset é também um clássico utilizado para testes de RNNs.

In [ ]:
train, train_info = tfds.load('imdb_reviews', split='train', with_info=True, batch_size=-1)
test, test_info = tfds.load('imdb_reviews', split='test', with_info=True, batch_size=-1)

x_train, y_train = train["text"], train["label"]
x_test, y_test = test["text"], test["label"]

Quando lidamos com dados sequenciais, devemos padronizar o tamanho das sequências para que todas tenham o mesmo número de símbolos. Isto se deve pelo fato das bibliotecas realizarem otimizações internas para que a execução do código seja mais veloz.

<br />

A forma correta de se fazer é selecionar um tamanho padrão para as sequências e, então, truncar as sequências maiores do que o limite ou adicionar um caractere *nulo* para preencher as sequências menores que o limite.


<br />

Pensando em facilitar a vida dos praticantes de *machine learning*, a biblioteca Keras nos fornece uma função auxiliar que permite fazer esta operação com poucas linhas de código. Da mesma forma, outras operações comuns ao processamento de textos também são implementadas no módulo `sequence` da biblioteca.

Abaixo, criamos um `layer` customizado que irá adptar as entradas em formato de texto para que possamos inserir na rede neural durante o treinamento. Note que a função `preprocess` também realiza a *limpeza* do texto, i.e., remove caracteres indesejados, como tags `HTML`. No caso do dataset já ter sido pré-processado e e estar "limpo", podemos apenas realizar o retorno do texto puro ou então tornar todas as letras minúsculas.

**Observação**: Decidir sobre o pré-processamento, trocar do *case* das letras, etc, faz parte do treino da rede neural recorrente para processamento de linguagem!

A classe `TextVectorization` irá criar `layer` customizado, propriamente dito. Devemos definir a quantidade máxima de palavras considerada no dicionário da rede neural (`max_tokens`) e o tamanho máximo de cada texto a ser tratado pela rede neural (`output_sequence_length`). Perceba também que o nosso pré-processamento é vinculado ao `layer` em `standardize`.

```
tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
    encoding='utf-8',
    **kwargs
)
```



In [ ]:
import re
import string

def preprocess(texto):
    lowercase = tf.strings.lower(texto)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=30000,
    standardize=preprocess,
    output_mode='int',
    output_sequence_length=80)
text_vectorizer.adapt(x_train)

O formato do dataset IMBD traz apenas a informação de que o *target* é positivo (1) ou negativo (0). Por isso, devemos modificar o dataset para que incluir tantas colunas quanto necessário de forma que cada *target* possua `N` colunas (uma para cada tipo de alvo), todas preenchidas com o valor 0. Em seguida, invertemos o valor da coluna correspondente ao *target* correto.

<br />

Esta operação se chama *One-hot encoding*. Note que esta operação não foi necessária para os exercícios sobre o *dataset* MNIST já que o mesmo nos foi fornecido pré-processado.

In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Vamos utilizar mais uma vez o modelo básico da biblioteca Keras que é o modelo `Sequential`. Este modelo nos permite adicionar camadas na rede neural que serão processadas na sequência em que forem inseridas. Ao criarmos um objeto deste tipo, podemos adicionar as camadas utilizando o método `add` do modelo.

<br/>

A primeira camada de qualquer modelo que faça processamento de textos é incluir uma camada chamada `Embedding`.

```
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    sparse=False
)
```

Esta camada irá codificar uma representação para as palavras que estão presentes no dataset. Para adicionar esta camada, devemos indicar a quantia de palavras que queremos incluir (também chamado de tamanho do vocabulário), e o tamanho desta representação (isto é, o número de dimenções para estas representações). Por exemplo:

```
tf.keras.layers.Embedding(1000, 32)
```

<br/>

Vimos também na aula de hoje as camadas recorrentes, que na biblioteca Keras são chamadas `SimpleRNN` e `LSTM` pela biblioteca Keras. Para cada camada, devemos informar o tamanho (número de unidades), e os valores de *dropout* e *recurrent_dropout* para a camada. De maneira geral, o *recurrent_dropout* é utilizado em casos em que a rede neural é muito grande, na casa de bilhões de parâmetros.

```
tf.keras.layers.SimpleRNN(
    units,
    activation='tanh',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros',
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    unroll=False,
    **kwargs
)

tf.keras.layers.LSTM(
    units,
    activation='tanh',
    recurrent_activation='sigmoid',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros',
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    time_major=False,
    unroll=False
)
```

<br />

Um detalhe de implementação: caso queiramos utilizar mais do que uma camada recorrente, precisamos setar `return_sequences=True` em todas as camadas recorrentes com exceção da última.

*Observação*: Ainda há a camadda `GRU`, que é uma simplificação da `LSTM`.

<br />

Outro detalhe que deve ser observado sobre as redes neurais recorrentes é o fato de que a inicialização de seus parâmetros difere das redes neurais *feedforward* (isto é, as redes neurais comuns) e das *Conv Nets*. De mandeira geral, inicializamos os parâmetros das RNNs selecionando-os em uma distribuição uniforme, onde todos os valores tem a mesma chance de ser selecionados, utilizando um intervalo reduzido (exemplos: $[-0.1, 0.1], [-0.05, 0.05]$, etc). Para isso, devemos importar o tipo de inicialização correta e passar a função como um parâmetro quando formos criar as camadas. A inicialização uniforme na biblioteca Keras é chamada de `RandomUniform`.

In [ ]:
from keras.models import Sequential

print('Build model...')

model = tf.keras.models.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(text_vectorizer)

model.add(tf.keras.layers.Embedding(30001, 32))

model.add(tf.keras.layers.SimpleRNN(64,
                                    use_bias=True,
                                    return_sequences=False, #altere para True quando tiver mais do que uma camada recorrente
                                    kernel_initializer= tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=42),
                                    recurrent_dropout=0.0,
                                    bias_initializer='zeros'))


model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.summary()

Antes de treinarmos o modelo, precisamos fazer sua compilação com o método `compile`. Isto utilizará a biblioteca tensorflow que fará diversas otimizações no código e irá gerar um executável na linguagem `C` (que não está imediatamente disponível para nós).

<br/>

Para compilar o código, precisamos passar qual a função de perda (`loss`), qual otimizador será utilizado para o treinamento (`optimizer`) e qual métrica será utilizada durante o treinamento para monitorarmos o progresso.

<br/>

A biblioteca keras oferece diversos otimizadores para treinamento. Para escolhermos, basta importar o otimizador desejado e fornecer para o método `compile`.

In [ ]:
from keras.optimizers import RMSprop, Adadelta, Adam, Adagrad

# aqui utilizaremos a função binary_crossentropy pois exatamente 2 classes e
# otimizador Adam enquanto monitoramos a evolução da acurácia para o modelo
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import seaborn as sns
sns.set()
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(18, 4))

# loss
ax1.plot(history.history['loss'])
ax1.plot(history.history['val_loss'])
ax1.set_title('Loss')
ax1.set(xlabel='epoch', ylabel='loss')
ax1.legend(['train', 'val'], loc='upper left')

# acurácia
ax2.plot(history.history['accuracy'])
ax2.plot(history.history['val_accuracy'])
ax2.set_title('Acurácia')
ax2.set(xlabel='epoch', ylabel='accuracy')
ax2.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.show()

A biblioteca Keras possui uma interface compatível com a biblioteca [scikit-learn](https://scikit-learn.org/stable/index.html), que é padrão para praticantes de *machine learning*. Por isso, podemos utilizar objetos e funções desta biblioteca para calcular métricas mais avançadas, como * precision*, * recall*  e * f1-score*.

<br/>

Note que a rede neural faz uma previsão do * score*  para cada classe possível para cada exemplo no dataset. Ou seja, para cada exemplo, a saída da rede neural será um *score* para cada classe possível. No nosso caso são 2 classes e por isso a saída da rede neural conterá 2 *scores* para cada exemplo.

<br/>

Em função desta saída em formato de scores, precisamos ober o score mais alto obtido para cada exemplo. Este * score*  irá nos dizer qual a classe foi prevista pela rede neural com um grau de confiança maior. Isso será obtido pela função `argmax` da biblioteca numpy

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(x_test)
print('Acurácia: {:.4f}'.format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

print('\n\nDemais métricas (separadas por classe): ')
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(x_test)
print('Acurácia: {:.4f}'.format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

print('\n\nDemais métricas (separadas por classe): ')
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

In [ ]:
frase1 = 'the movie was great.'

pred = model.predict(tf.convert_to_tensor([f'{frase1}'], dtype=tf.string) )

In [ ]:
# insira uma frase de teste abaixo
frase1 = 'it was a really awesome movie'

pred = model.predict(tf.convert_to_tensor([frase1], dtype=tf.string) )
print(f'Probabilidade negativo: {(pred[0][0] * 100):.4f}% \nProbabilidade positivo: {(pred[0][1] * 100):.4f}%')